Problem 3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, t
from statsmodels.tsa.arima.model import ARIMA
from scipy.optimize import minimize

In [19]:
# Implement return_calculate
def return_calculate(prices, method="DISCRETE", dateColumn="Date"):
    vars_ = prices.columns
    nVars = len(vars_)
    vars_ = [var for var in vars_ if var != dateColumn]
    if nVars == len(vars_):
        raise ValueError(f"dateColumn: {dateColumn} not in DataFrame: {vars_}")
    nVars = nVars - 1
    p = prices[vars_].to_numpy()
    n, m = p.shape
    p2 = np.empty((n-1, m))
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in (\"LOG\",\"DISCRETE\")")
    dates = prices[dateColumn].iloc[1:n].to_numpy()
    out = pd.DataFrame({dateColumn: dates})
    for i in range(nVars):
        out[vars_[i]] = p2[:, i]
    return out

In [20]:
# Import data, get the price for each asset in portfolio and calculate the current price
portfolio = pd.read_csv("portfolio.csv")
prices = pd.read_csv("DailyPrices.csv")
def get_portfolio_price(portfolio, prices, portfolio_name, Delta=False):
    if portfolio_name == "All":
        assets = portfolio.drop('Portfolio',axis=1)
        assets = assets.groupby(["Stock"], as_index=False)["Holding"].sum()
    else:
        assets = portfolio[portfolio["Portfolio"] == portfolio_name]     
    stock_codes = list(assets["Stock"])
    assets_prices = pd.concat([prices["Date"], prices[stock_codes]], axis=1) 
    current_price = np.dot(prices[assets["Stock"]].tail(1), assets["Holding"])
    holdings = assets["Holding"]   
    if Delta == True:
        asset_values = assets["Holding"].values.reshape(-1, 1) * prices[assets["Stock"]].tail(1).T.values
        delta = asset_values / current_price       
        return current_price, assets_prices, delta   
    return current_price, assets_prices, holdings

In [21]:
# Using exponentially weighted covariance with lambda=0.97
def exp_weighted_cov(returns, lambda_=0.97):
    returns = returns.values
    mean_return = np.mean(returns, axis=0)
    normalized_returns = returns - mean_return

    n_timesteps = normalized_returns.shape[0]
    cov = np.cov(returns, rowvar=False)

    for t in range(1, n_timesteps):
        cov = lambda_ * cov + (1 - lambda_) * np.outer(normalized_returns[t], normalized_returns[t])
    return cov

In [22]:
# Calculate with Delta Normal
def calculate_delta_var(portfolio, prices, alpha=0.05, lambda_=0.94, portfolio_name="All"):
    current_price, assets_prices, delta = get_portfolio_price(portfolio, prices, portfolio_name, Delta=True)
    returns = return_calculate(assets_prices).drop('Date', axis=1)
    assets_cov = exp_weighted_cov(returns, lambda_)
    p_sig = np.sqrt(np.transpose(delta) @ assets_cov @ delta)
    var_delta = -current_price * norm.ppf(alpha) * p_sig
    return current_price[0], var_delta[0][0]

In [23]:
# Calculate with historical simulation
def calculate_historic_var(portfolio, prices, alpha=0.05,n_simulation=1000, portfolio_name="All"):
    current_price, assets_prices, holdings = get_portfolio_price(portfolio, prices, portfolio_name)  
    returns = return_calculate(assets_prices).drop("Date", axis=1)    
    assets_prices = assets_prices.drop('Date',axis=1)
    sim_returns = returns.sample(n_simulation, replace=True)
    sim_prices = np.dot(sim_returns* assets_prices.tail(1).values.reshape(assets_prices.shape[1],),holdings)   
    var_hist = -np.percentile(sim_prices, alpha*100) 
    return current_price[0], var_hist, sim_prices

In [37]:
# Print the results
current_price, delta_var = calculate_delta_var(portfolio, prices, portfolio_name='A')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, prices, portfolio_name='A')
print("The current value for A is: {:.2f}".format(current_price))
print("VaR for A using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for A using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for A is: 299950.06
VaR for A using Delta Normal is: 5670.20
VaR for A using Historic Simulation is: 7109.30



In [38]:
current_price, delta_var = calculate_delta_var(portfolio, prices, portfolio_name='B')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, prices, portfolio_name='B')
print("The current value for B is: {:.2f}".format(current_price))
print("VaR for B using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for B using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for B is: 294385.59
VaR for B using Delta Normal is: 4494.60
VaR for B using Historic Simulation is: 7273.77



In [39]:
current_price, delta_var = calculate_delta_var(portfolio, prices, portfolio_name='C')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, prices, portfolio_name='C')
print("The current value for C is: {:.2f}".format(current_price))
print("VaR for C using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for C using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for C is: 270042.83
VaR for C using Delta Normal is: 3786.59
VaR for C using Historic Simulation is: 5310.07

